In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Step 1: Read the preprocessed data
data_path = 'AI_Human.csv'
df = pd.read_csv(data_path)
print("data loaded. Shape:", df.shape)

data loaded. Shape: (487235, 2)


In [3]:
# Step 2: Sample 20,000 from each class (or change to 100 for testing)
n_samples_per_class = 10000  # Set to 100 if you want a tiny dataset
df_human = df[df['generated'] == 0].sample(n=n_samples_per_class, random_state=42)
df_ai = df[df['generated'] == 1].sample(n=n_samples_per_class, random_state=42)
df_sampled = pd.concat([df_human, df_ai]).sample(frac=1, random_state=42).reset_index(drop=True)
print("Sampled data shape:", df_sampled.shape)

Sampled data shape: (20000, 2)


In [4]:
# Step 3: Preprocess the sampled data
df_sampled = df_sampled.dropna(subset=['text'])
df_sampled['generated'] = df_sampled['generated'].astype(int)
print("Shape after preprocessing (removing NaN):", df_sampled.shape)
print("Unique values in 'generated':", df_sampled['generated'].unique())
df_sampled.to_csv('sampled_preprocessed_data.csv', index=False)
print("Sampled preprocessed data saved as 'sampled_preprocessed_data.csv'")

Shape after preprocessing (removing NaN): (20000, 2)
Unique values in 'generated': [1 0]
Sampled preprocessed data saved as 'sampled_preprocessed_data.csv'


In [5]:
# Step 4: Split into train (80%), validation (10%), test (10%)
train_df, temp_df = train_test_split(df_sampled, test_size=0.2, random_state=42, stratify=df_sampled['generated'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['generated'])
print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

Train set shape: (16000, 2)
Validation set shape: (2000, 2)
Test set shape: (2000, 2)


In [6]:
# Step 5: Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [7]:
# Step 6: Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
# Step 7: Set format for PyTorch and verify labels
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'generated'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'generated'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'generated'])

# Rename 'generated' to 'labels'
train_dataset = train_dataset.rename_column("generated", "labels")
val_dataset = val_dataset.rename_column("generated", "labels")
test_dataset = test_dataset.rename_column("generated", "labels")

# Verify label tensor shape and type
print("Train dataset labels sample:", train_dataset['labels'][:5])
print("Train labels dtype:", train_dataset['labels'].dtype)

Train dataset labels sample: tensor([1, 0, 0, 1, 1])
Train labels dtype: torch.int64


In [9]:
# Step 8: Load DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Step 9: Define compute_metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

In [11]:
# Step 10: Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Increased for faster training
    per_device_eval_batch_size=16,
    warmup_steps=200,  # Reduced since model is smaller
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
)

d:\python-3.10\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Step 11: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
# Step 12: Train the model
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.045600,0.095513,0.980500
2,0.000200,0.047782,0.991000
3,0.000200,0.044495,0.992500


TrainOutput(global_step=3000, training_loss=0.04715014288151481, metrics={'train_runtime': 84409.944, 'train_samples_per_second': 0.569, 'train_steps_per_second': 0.036, 'total_flos': 6358435135488000.0, 'train_loss': 0.04715014288151481, 'epoch': 3.0})

In [14]:
# Step 13: Save the best model
model.save_pretrained('./best_distilbert_model')
tokenizer.save_pretrained('./best_distilbert_model')
print("Best model saved to './best_distilbert_model'")

Best model saved to './best_distilbert_model'


In [15]:
# Step 14: Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(test_dataset)
print("Test set accuracy:", test_results['eval_accuracy'])

# Detailed classification report
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids
print("\nClassification Report on Test Set:")
print(classification_report(true_labels, pred_labels, target_names=['Human', 'AI']))

Evaluating on test set...


Test set accuracy: 0.99

Classification Report on Test Set:
              precision    recall  f1-score   support

       Human       1.00      0.98      0.99      1000
          AI       0.98      1.00      0.99      1000

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [23]:
# Import necessary libraries
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Step 1: Load the saved model and tokenizer
model_path = 'best_distilbert_model'  # Path where your 99% model is saved
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Move model to appropriate device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()  # Set to evaluation mode

# Step 2: Provide your 1,000-word text
# Replace this with your actual 1,000-word text
new_text = """
iyad and, mohand go to play football this day, then i will watch match of football this day on
"""

# Step 3: Tokenize the new text
inputs = tokenizer(new_text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
# Move inputs to the same device as the model
inputs = {key: val.to(device) for key, val in inputs.items()}

# Step 4: Run inference
with torch.no_grad():  # No gradient computation for inference
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)  # Convert logits to probabilities
    predicted_class = torch.argmax(probabilities, dim=-1).item()  # Get predicted class (0 or 1)

# Step 5: Interpret and print results
class_names = ['Human', 'AI']
predicted_label = class_names[predicted_class]
confidence = probabilities[0][predicted_class].item() * 100  # Confidence percentage

print(f"Predicted class: {predicted_label}")
print(f"Confidence: {confidence:.2f}%")
print(f"Probabilities - Human: {probabilities[0][0].item()*100:.2f}%, AI: {probabilities[0][1].item()*100:.2f}%")

# Step 6 (Optional): If you know the true label, calculate accuracy
true_label = 0  # Replace with 0 (Human) or 1 (AI) if you know the ground truth
if true_label is not None:
    accuracy = 1.0 if predicted_class == true_label else 0.0
    print(f"True label: {class_names[true_label]}")
    print(f"Accuracy for this sample: {accuracy*100:.0f}%")
else:
    print("No true label provided, accuracy cannot be calculated.")

Predicted class: Human
Confidence: 98.59%
Probabilities - Human: 98.59%, AI: 1.41%
True label: Human
Accuracy for this sample: 100%
